<a href="https://colab.research.google.com/github/v5v5/data-science-plan/blob/main/ChatBot_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
with open('/content/BIG_BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

with open('/content/test.txt', 'w') as f:
   f.write('test text')

In [ ]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Хай!', 'Добрый день'],
#             'resposes': ['Доброго времени суток', 'Прив', 'Хаюхай']
#         },
#         'bye': {
#             'examples': ['Пока', 'До свидания', 'Увидимся'],
#             'resposes': ['Счатливо', 'Удачного дня', 'Приходите еще']
#         }
#     }
# }

In [ ]:
X = []
y = []

for intent, value in BOT_CONFIG['intents'].items():
    if 'inc_examples' in value:
      examples = list(set([example.lower() for example in value['inc_examples']]))
    else:
      examples = list(set([example.lower() for example in value['examples']]))
    X += examples
    y += [intent] * len(examples)

vectorizer = TfidfVectorizer()
X_transformed = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
def clean(text):
  return ''.join([simbol for simbol in text.lower() if simbol in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '])

def match(example, text):
  return nltk.edit_distance(clean(text), clean(example)) / len(example) < 0.4

def get_intent(text):
  for intent, value in BOT_CONFIG['intents'].items():
    for example in value['examples']:
      if match(example, text):
        return intent

def get_intent_by_ml_model(text):
  return classifier.predict(vectorizer.transform([text]))[0]

In [ ]:
question = input()
answer = get_intent(question)
print(answer)

хай
hello


In [ ]:
question = ''
while question != 'выход':
  question = input()

  intent = get_intent_by_ml_model(question)
  print(random.choice(BOT_CONFIG['intents'][intent]['responses']))

привет
приветствую
как дела?
Все ОК
где ты живешь?
То там, то тут
выход
Приходите еще


In [ ]:
def bot(question):
    intent = get_intent_by_ml_model(question)
    return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
question = ''
while question != 'выход':
  question = input()
  print(bot(question))

прив
Здравствуйте
хай
Доброго времени суток
пока
Удачного дня
выход
Удачного дня


In [ ]:
# !pip install python-telegram-bot --upgrade

In [ ]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)
questions = []

# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""
    question = update.message.text
    questions.append(question)
    reply = bot(question)
    # reply = 'https://colab.research.google.com/drive/1Azj_M62pXee_7DlknRx7nEqEB2j7SoFp#scrollTo=zuaT9rmGkjZw'
    update.message.reply_text(reply)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1753323831:AAGUpWW1cbCxVUDWFO_N77KUeFnStUsLxkU")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()